In [8]:
import pandas as pd
label_df = pd.read_csv("/home/kongge/projects/Phd/data/all_labels.csv")
prs_df = pd.read_csv('/home/kongge/projects/Phd/data/Only_PGS.csv')

In [9]:
data = pd.merge(prs_df, label_df, left_on='FID', right_on='eid')

In [10]:
data.shape

(336979, 187)

In [11]:
X = prs_df.drop(columns=['FID'])

In [12]:
X.shape

(336979, 80)

In [13]:
y = label_df.drop(columns=['eid'])

In [14]:
y.shape

(336979, 105)

In [18]:
y.isnull().sum().sum()

0

In [20]:
pip install --upgrade imbalanced-learn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.8 MB/s eta 0:00:0000:01
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
X_train

,AF,CAD,DCM,HT,ISS,VTE,STK,HF,IH,AAA,...,GM,HB,HIR,IRM,ISM,NMR,PM,TSH,TM,AST
333727,0.822660,0.652212,1.015069,0.157939,0.924092,-0.405650,0.349883,-0.087631,-2.038007,0.598120,...,1.168651,-0.884286,0.434311,0.229408,1.133883,-1.071869,1.231342,-2.165018,0.240285,1.506544
200703,1.315091,2.690288,-0.103229,2.790996,-1.341288,-1.473949,1.042497,-1.580591,0.975190,0.069414,...,-1.354292,-2.640484,-2.271821,-1.302221,1.817624,0.038104,0.644183,1.182140,-1.037645,-0.628436
44890,0.109502,0.102083,3.056032,-1.240268,1.387456,-0.708312,-1.332807,1.057653,-2.138872,1.772844,...,0.743734,0.571942,1.792547,-0.306317,1.619743,0.565118,1.890026,0.106394,1.303391,-0.616528
198338,-0.846815,-0.152581,0.858531,0.625978,-1.295693,-0.938884,-1.064840,-0.646639,1.819375,0.261589,...,-1.433963,-0.484939,-2.590601,0.123784,0.701905,0.038104,-0.097592,0.085232,-1.577460,1.167762
155891,1.370694,-1.279699,1.157835,-0.997492,-0.407646,0.735023,-2.681439,2.666505,0.267222,0.321081,...,-1.194948,0.399326,0.159918,-0.018817,0.260539,0.038104,-0.343630,0.414197,-1.445260,0.383889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,-0.822246,1.309619,-0.619525,2.514850,-1.477214,1.663976,0.451371,-0.094448,0.692296,-1.159496,...,1.009307,0.099106,-0.976158,0.902270,-1.593487,0.038104,-0.108222,2.607063,0.008936,-0.456354
259178,0.914394,0.284413,1.558108,-0.044281,2.395993,0.452445,-0.007772,1.555307,0.594953,-1.028196,...,0.770291,0.095480,1.026520,-0.998043,-0.497368,0.565118,-0.124039,-0.270999,-0.988070,0.074991
131932,-0.983418,0.505429,0.555912,-0.825622,0.320348,-0.918190,-1.104035,-1.430613,1.396819,0.099803,...,2.469958,-1.347018,-0.488784,-0.963222,-1.018091,-1.071869,0.295106,-2.722289,1.226275,1.853827
146867,0.692566,0.159414,1.260993,0.222677,0.309454,1.237205,0.846188,-1.628311,-0.044726,-0.779552,...,-1.274620,-0.065079,-0.487734,-0.593957,0.007281,0.565118,2.103121,-2.658803,1.171415,0.471001


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
aupr_results = {}

# 针对每个疾病（标签）训练独立的随机森林分类器
for disease in y_train.columns:
    print(f"正在训练疾病分类器：{disease}")

    rf_classifier = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train[disease])
    y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]
    aupr = average_precision_score(y_test[disease], y_pred_proba)
    aupr_results[disease] = aupr
    print(f"{disease} 的 AUPR: {aupr:.4f}")

正在训练疾病分类器：Class_A00
Class_A00 的 AUPR: 0.0000
正在训练疾病分类器：Class_A01
Class_A01 的 AUPR: 0.0002
正在训练疾病分类器：Class_A02
Class_A02 的 AUPR: 0.0015
正在训练疾病分类器：Class_A03
Class_A03 的 AUPR: 0.0005
正在训练疾病分类器：Class_A04


In [ ]:
aupr_df = pd.DataFrame(list(aupr_results.items()), columns=['Disease', 'AUPR']).sort_values(by='AUPR', ascending=False)

output_file = "aupr_results.csv"
aupr_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"AUPR 结果已写入文件：{output_file}")